# Homework 1 - KMeans Clustering
For homework 1, you will apply kmeans clustering to real environmental datasets. This assignment is designed to provide hands-on practices. 

MAKE YOUR OWN COPY OF THIS FILE BEFORE YOU START. 

Complete each task and submit your Jupyter notebook on Blackboard.

# Section:
- Random Initialization
- Assign Points
- Compute Centroids
- KMeans Clustering
- Silhouette Score
- Real-World Applications
  - Climate
  - Wildfire (MS Student Only)

## To-Do Lists
Look out for sections marked "# IMPLEMENT" and "# QUESTION"
- Undergrads: 4 Implement Blocks + 1 Question Block - 5 Points Total
- Masters: 6 Implement Blocks + 1 Question Block - 7 Points Total

Partial credits will be given.




--- 





In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
np.random.seed(500)

In [ ]:
# randomly generated example data to cluster
example_X = np.concatenate([np.random.multivariate_normal((3,3),((1,0),(0,1)), 30), 
                           np.random.multivariate_normal((-3,3),((1,-0.5),(-0.5,1)), 30), 
                           np.random.multivariate_normal((3,-3),((1,0),(0,1)), 30)])

: 

In [ ]:
print(example_X.shape)

: 

In [ ]:
# example data visualization
plt.scatter(example_X[:,0], example_X[:,1])
plt.show()

## [1] Random Initialization

In [ ]:
""" 

Randomly initialize cetroids for kmeans clustering based on points X.
X is passed in as a numpy array. You should randomly choose k different
points from X.

Output should be a numpy array of size KxM where K is the number of centroids
and M is the number of features.

"""

def random_initialization(X: np.array, k: int):
    # -------------------------------------------------------------------------
    # IMPLEMENT - 1 Point
    # -------------------------------------------------------------------------
    initial_centroids = X.copy()
    np.random.shuffle(initial_centroids)
    return initial_centroids[:k]

## [2] Assign Points

In [ ]:
"""

Given dataset X, "cluster" (assign) each point to be the centroid that is closest
according to euclidean distance, you can implement euclidean distance yourself
or use sklearn's implementation.

Output should be a numpy array of shape NxM where n is the number of points and 
m is the number of features.

"""

from sklearn.metrics.pairwise import euclidean_distances

def assign_points(X: np.array, centroids: np.array):
    # -------------------------------------------------------------------------
    # IMPLEMENT - 1 Point
    # -------------------------------------------------------------------------
    point_assignments = np.empty([len(X), X.shape[1]])
    allCentroidDistances = euclidean_distances(X, centroids)
    for i in range(len(allCentroidDistances)):
        row = allCentroidDistances[i]
        minVal = np.iinfo(np.int32).max
        currentPoint = row
        for cell in row:
            if cell < minVal:
                minVal = cell
        result = np.where(currentPoint == minVal)
        point_assignments[currentPoint] = result[0]
    return point_assignments

## [3] Compute Centroids

In [ ]:
"""

Given dataset X (size NxM), and cluster assignments (NxM), compute centroids of each cluster
according to the given point assignments.

Output should be a numpy array of shape (KxM) 
where K is the number of clusters, and M is the number of features.

"""

def compute_centroids(X: np.array, point_assignments: np.array, k: int):
    # -------------------------------------------------------------------------
    # IMPLEMENT - 1 Point
    # -------------------------------------------------------------------------
    #3*k sums... one for each running total of feature position of points corresponding to each centroid (x,y)
    #one for the number of points corresponding to each centroid
    #then average the feature sums for each centroid (divide by num points sum) to yield k new x,y positions 
    centroids = np.empty([k, X.shape[1]])
    while k > 0:
        for a in point_assignments:
            result = np.where(a==k)
            #ksum = sum(X[result[0]])
            #kcount = len(result[0])
            #kpos = ksum / kcount
            kpos = np.mean(X[result[0]], axis=0)
            centroids[k] = kpos
        k=k-1
    return centroids

## [4] KMeans Clustering

In [ ]:
"""

Given dataset X and number K, cluster points into K categories using K-Means clustering.

To do this you will need to initialize cluster centers
repeat until cluster centers are not changed:
    update point assignments to clusters
    update cluster centers

This should return clusters as a numpy array (Nx1) with one entry for each point in X, 
with entries corresponding to cluster ids from 0 to K-1.

"""

from mimetypes import init


def KMeans_cluster(X: np.array, k: int):
    # -------------------------------------------------------------------------
    # IMPLEMENT - 1 Point
    # -------------------------------------------------------------------------
    initCentroids = random_initialization(X, k)
    x =1
    while(x == 1):
        assign_points(X, initCentroids)
        prevCentroids = initCentroids
        initCentroids = compute_centroids(X, initCentroids, k)
        if(prevCentroids.all() == initCentroids.all()):
            break

    final_cluster_assignments = initCentroids
    return final_cluster_assignments

Now we can use the clustering code to do interesting things like determine homogeneous bio-regions. Let's test it first on our example dataset

In [ ]:
cluster_assignments = KMeans_cluster(X = example_X, k = 3)

In [ ]:
plt.scatter(example_X[:,0], example_X[:,1], c=cluster_assignments)
plt.show()

## [5] Real-World Application - Climate
Run all of these steps to see eco-regions from raw data.
The raw data is geospatial data in the form of 'rasters', a grid overlay over a geographic extent where each cell is associated with one or more values. In our case each 10 by 10 meters grid cell across the world is associated with a set of environmental variables such as temperatures and precipitation. In fact, we have 3 sets of data one for the present environmental values of each grid cell, and two for future projected climates under two different climate change scenarios rcp2.6 and rcp8.5 in year 2050. This is the well known WORLDCLIM dataset (https://www.worldclim.org/data/index.html)

In [ ]:
!pip install rasterio
!wget -Nq http://biogeo.ucdavis.edu/data/climate/worldclim/1_4/grid/cur/bio_10m_esri.zip

!wget -Nq http://biogeo.ucdavis.edu/data/climate/cmip5/10m/bc85bi50.zip
!wget -Nq http://biogeo.ucdavis.edu/data/climate/cmip5/10m/bc26bi50.zip

!unzip -nq bio_10m_esri.zip
!unzip -nq bc85bi50.zip -d projection_data
!unzip -nq bc26bi50.zip -d projection_data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%matplotlib inline
import os             # do cross platform manipulation of filesystem

import numpy as np    # matrix library
import scipy
import scipy.spatial  # for distance calculations

import rasterio       # read in and handle rasters

import matplotlib     # plotting library
import matplotlib.pyplot as plt

import seaborn as sns # plotting library expansion

from sklearn import preprocessing  # for preprocessing and storing data transformations
from sklearn import metrics # for quantitatively evaluating our ml models
from scipy import stats     # statistics library for describing data

In [ ]:
# ordered list of feature names found here: http://www.worldclim.org/bioclim
feature_names = [
  'Annual Mean Temperature',
  'Mean Diurnal Range (Mean of monthly (max temp - min temp))',
  'Isothermality (BIO2/BIO7) (* 100)',
  'Temperature Seasonality (standard deviation *100)',
  'Max Temperature of Warmest Month',
  'Min Temperature of Coldest Month',
  'Temperature Annual Range (BIO5-BIO6)',
  'Mean Temperature of Wettest Quarter',
  'Mean Temperature of Driest Quarter',
  'Mean Temperature of Warmest Quarter',
  'Mean Temperature of Coldest Quarter',
  'Annual Precipitation',
  'Precipitation of Wettest Month',
  'Precipitation of Driest Month',
  'Precipitation Seasonality (Coefficient of Variation)',
  'Precipitation of Wettest Quarter',
  'Precipitation of Driest Quarter',
  'Precipitation of Warmest Quarter',
  'Precipitation of Coldest Quarter'
]

# format strings that map us to the files we want
# using os.path.join to get platform independent paths
current_feat_file_name_fmt = os.path.join("bio","bio_{feat_ind}","hdr.adf")
future_feat_file_name_fmt = os.path.join("projection_data","{model}{feat_ind}.tif")

model_names = ["bc26bi50","bc85bi50"]

current_filenames = [current_feat_file_name_fmt.format(feat_ind=feat_ind) for feat_ind in range(1, len(feature_names)+1)]

# dictionary (hashtable) mapping a model name to a list of feature names
# we can look up the feature names for a given model name by doing
# future_filenames["model"], take a look at 
# future_filenames.keys() to see what models there are
future_filenames = {model:[future_feat_file_name_fmt.format(model=model, feat_ind=feat_ind) for feat_ind in range(1, len(feature_names)+1)] for model in model_names}

In [ ]:
def load_features(file_names):
    
    """

    Load masked rasters from a list of filenames
    Loads all the data in the given input filenames and masks away any square in the raster that does not have an observation for "all" of the features
    
    Args:
        file_names (str list) : List of filenames from which to read individual features
    
    Returns:
        data (numpy masked array) : numpy matrix excluding entries with no data in any feature
    
    """
    
    mask = None
    raw_data = []
    for file_name in file_names:    
        # open raster for reading the raw data
        with rasterio.open(file_name, "r") as f:
            data = f.read().squeeze().astype(np.float32)
        raw_data.append(data)
        if mask is None:
            mask = (data == f.nodata)
        else:
            mask = mask | (data == f.nodata)
    
    # mask away cells for which we have no data
    data = np.ma.array(np.stack(raw_data, axis=-1),mask=np.repeat(mask[:,:,None],len(file_names), axis=-1))
    
    return data

In [ ]:
def show_map(data, title=None):
    
    """
    
    Plots data for a scalar map
    
    Args:
        data (numpy array) : raster of values to plot
    
    """
    
    plt.figure(figsize=(10,7))
    plt.imshow(data, cmap="Blues")
    plt.colorbar(fraction=0.03, pad=0.04)
    plt.grid("off")
    if title is not None:
        plt.title(title, fontsize=16)
    plt.show()
    plt.close()

In [ ]:
current_features = load_features(current_filenames)
projected_features = {model: load_features(model_files) for model, model_files in future_filenames.items()}

# create an empty raster with the same size and mask to plot useful things later
empty_raster = np.ma.array(np.zeros_like(current_features[:,:,0]),mask=current_features.mask[:,:,0])

In [ ]:
def flatten_mask(data):
    return data[~data.mask[:,:,0],:]

In [ ]:
current_observations = flatten_mask(current_features)
projected_observations = {model: flatten_mask(projected_features[model]) for model, model_files in future_filenames.items()}

In [ ]:
print("current indicators shape:",current_observations.shape)
print("best case scenario shape:",projected_observations["bc26bi50"].shape)

Transform and rescale features.

In [ ]:
def log_preprocess(data, log_features=None):
    res = data.copy()
    if log_features is not None:
        res[:,log_features] = np.log(res[:,log_features] + 1)
    return res

In [ ]:
def log_inv(data, log_features=None):
    res = data.copy()
    if log_features is not None:
        res[:,log_features] = np.exp(res[:log_features]) - 1
    return res

In [ ]:
log_features = None
log_features = [11,12,13,14,15,16,17,18]

log_transform = preprocessing.FunctionTransformer(func=log_preprocess,inverse_func=log_inv,kw_args={"log_features":log_features},check_inverse=False)

In [ ]:
scaler = preprocessing.StandardScaler(copy=True)

In [ ]:
processed_current_observations = scaler.fit_transform(log_transform.fit_transform(current_observations))
processed_projections = {model: scaler.transform(log_transform.fit_transform(model_observations)) for model, model_observations in projected_observations.items()}

In [ ]:
def plot_cluster(labels, title=None, empty_raster=empty_raster):
    label_raster = empty_raster.copy()
    label_raster[~label_raster.mask] = labels
    plt.figure(figsize=(18,6))
    im = plt.imshow(label_raster, cmap='tab20', vmin=0, vmax=labels.max())
    plt.colorbar(im, fraction=0.046, pad=0.04)
    if title is not None:
        plt.title(title, fontsize=16)
    plt.grid("off")

In [ ]:
%%time
training_data = np.concatenate([processed_current_observations, processed_projections["bc26bi50"]])

# Cluster on climate data - KMeans used here!

Note: If you cannot get your own implementation of K-Means clustering to work, replace the call to your function below with calling the sklearn implementation of batched K-Means and demonstrating results for several number of clusters, and checking how regions change based on number of clusters (for some credit).

In [ ]:
current_labels = KMeans_cluster(processed_current_observations, k=3)

In [ ]:
plot_cluster(current_labels, "clustering of current data")

In [ ]:
'''

Q: Are Canada and Europe in the same cluster?

'''

# -------------------------------------------------------------------------
# QUESTION - 1 Point
# -------------------------------------------------------------------------

# Your answer.

Optional Exploration: Let us below rerun the the clustering with same K (but different initializations because of the randomness), what happens to the results? Next rerun the clustering for larger K=4,6,8, and comment on what you observe with the resulting maps, what would be your recommended K based on this manual inspection?

In [ ]:
k=4
current_labels = KMeans_cluster(processed_current_observations, k)
plot_cluster(current_labels, "clustering of current data")

OPTIONAL: share your observations here in text comments.

##  [6] Real-World Application - Wildfire (MS Student Only)

The Fire Program Analysis Fire-Occurrence Database (FPA FOD) includes 1.88 million geo-referenced wildfire records from 1992 to 2015. (https://www.kaggle.com/datasets/rtatman/188-million-us-wildfires). Wildfire.csv is generated by filtering FPA FOD: (1) wildfire size class, wildfire year, and state; (2) 2001 - 2015; (3) 50 states. Extract the largest wildfire class for each year in each state and map wildfire size class from letters to numbers (ex. Class A => 0, Class B => 1, so on...). 

In [ ]:
!wget -Nq https://raw.githubusercontent.com/csci461/dataset/main/wildfire.csv

In [ ]:
import plotly.express as px
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

FIRE_SIZE_CLASS = Code for fire size based on the number of acres within the final fire perimeter expenditures (A=greater than 0 but less than or equal to 0.25 acres, B=0.26-9.9 acres, C=10.0-99.9 acres, D=100-299 acres, E=300 to 999 acres, F=1000 to 4999 acres, and G=5000+ acres). Map wildfire size class from letters to numbers (ex. Class A => 0, Class B => 1, so on...). 

In [ ]:
df = pd.read_csv("wildfire.csv",index_col=0)
df.head()

In [ ]:
'''

Use sklearn KMeans (set random state to 123, use default hyperparameters) and silhouette_score

Cluster 50 states by wildfire size class from 2001 to 2015

Use silhouette score to determine top 2 number of clusters (M clusters, N clusters)

Show line plot - number of clusters (from 2 to 19) vs silhouette score 

'''

# -------------------------------------------------------------------------
# IMPLEMENT - 1 Point
# -------------------------------------------------------------------------


In [ ]:
def cluster_map(df,cluster_label,title):
  
    """

    Plot cluster labels on 50 states map
    
    Args:
        df (pandas dataframe) : dataframe with states as index and year as columns
        cluster_label (numpy array) : cluster labels for 50 states
        title (str) : title for the plot

    Returns:
        data (numpy masked array) : numpy matrix excluding entries with no data in any feature
    
    """

    df_cluster = df.copy()
    df_cluster["Cluster"] = [str(item) for item in list(cluster_label)]
    fig = px.choropleth(df_cluster,locations = df_cluster.index, locationmode="USA-states", scope="usa",
                      color="Cluster",category_orders={"Cluster": np.sort(df_cluster["Cluster"].unique())},
                      title=title)
    fig.show()

In [ ]:
'''

Cluster 50 states by wildfire size class from 2001 to 2015 with the best number of clusters M

Show cluster map

Cluster 50 states by wildfire size class from 2001 to 2015 with the 2nd best number of clusters N

Show cluster map 

'''

# -------------------------------------------------------------------------
# IMPLEMENT - 1 Point
# -------------------------------------------------------------------------

cluster_map(df,kmeans_labels_for_cluster_M,"2000 - 2015 Largest Wildfire Size Class")

cluster_map(df,kmeans_labels_for_cluster_N,"2000 - 2015 Largest Wildfire Size Class")

In [ ]:
from IPython.display import Image
Image(url="https://hazards.fema.gov/nri/Content/Images/StaticPageImages/map-wildfire_risk.png", width=600, height=300)

In [ ]:
'''

Q: FEMA provides a wildfire risk map (see above). Look back at your cluster maps. 
   Which one is the best match (visually) to the FEMA map? How many clusters?

'''

# -------------------------------------------------------------------------
# QUESTION - 1 Point
# -------------------------------------------------------------------------

# Your answer.